In [1]:
import pandas as pd

In [2]:
pbp_data = pd.read_csv('data/game_data/2014111300_plays.csv')

In [3]:
pbp_data.sort_values(by=['drive','play'], axis=0, inplace=True)

In [4]:
pbp_data.head()

,drive,play,down,time,desc,ydstogo,qtr,ydsnet,yrdln,sp,posteam,note
135,1,36,0,15:00,NaN,0,1,13,BUF 35,0,BUF,KICKOFF
118,1,69,1,14:47,NaN,10,1,13,MIA 12,0,MIA,NaN
132,1,93,1,14:26,NaN,10,1,15,MIA 25,0,MIA,NaN
127,1,117,2,13:49,NaN,8,1,25,MIA 27,0,MIA,NaN
128,1,146,1,13:14,NaN,10,1,27,MIA 37,0,MIA,NaN


#### Getting the point_spread data

In [5]:
fav = 'MIA'

In [6]:
spread = -4

In [7]:
pbp_data['scorediff'] = spread

In [8]:
cur_scorediff = spread
for index,row in pbp_data.iterrows():
    if row['sp'] == 1:
        if row['note'] == 'TD':
            if row['posteam'] == fav:
                cur_scorediff += 6
            else:
                cur_scorediff -= 6
        if row['note'] == 'FG' or row['note'] == 'SAF':
            if row['posteam'] == fav:
                cur_scorediff += 3
            else:
                cur_scorediff -= 3
        if row['note'] == 'XP':
            if row['posteam'] == fav:
                cur_scorediff += 1
            else:
                cur_scorediff -= 1
    pbp_data.loc[index,'scorediff'] = cur_scorediff    

#### Generating the Y axis i.e. was the bet cleared or not

In [9]:
pbp_data['betcleared'] = True

In [10]:
pbp_y = pbp_data['betcleared']
pbp_x = pbp_data.drop(['betcleared'], axis=1)

#### Now filtering the data

In [20]:
colnames_x = ['down','scorediff', 'ydstogo','betcleared']

In [21]:
from sklearn import svm

In [39]:
clf = svm.SVC()

In [40]:
pbp_X = pbp_data[colnames_x].dropna(axis=0,how='any')

In [41]:
pbp_Y = pbp_X['betcleared']
pbp_X = pbp_X.drop(['betcleared'], axis=1)

filling half of the Y array with false

In [42]:
pbp_Y.iloc[75:] = False

In [43]:
clf.fit(pbp_X, pbp_Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [47]:
predi_Y = clf.decision_function(pbp_X)

In [49]:
classes = clf.predict(pbp_X)

In [56]:
predi_Y[65:75]

array([-0.9999092 , -1.00017479,  0.50364045, -0.9999092 , -0.06023438,
        0.31208934,  1.00018783,  1.00020594, -0.9999092 ,  0.31208934])

In [57]:
classes[65:75]

array([False, False,  True, False, False,  True,  True,  True, False,  True], dtype=bool)